# செமாண்டிக் கர்னலில் ஏஜென்ட் ஸ்க்ராட்ச்பேடுடன் உரையாடல் வரலாற்றை குறைப்பது

இந்த நோட்புக் செமாண்டிக் கர்னலின் உரையாடல் வரலாற்றை குறைக்கும் அம்சத்தையும், ஏஜென்ட் ஸ்க்ராட்ச்பேடுடன் உரையாடல்களில் நிலையான சூழலை பராமரிக்க பயன்படுத்துவது எப்படி என்பதை விளக்குகிறது. இது நீண்ட உரையாடல்களை கையாளும் திறன் கொண்ட திறமையான AI ஏஜென்ட்களை உருவாக்க முக்கியமானது, குறிப்புகளின் வரம்புகளை மீறாமல்.

## நீங்கள் கற்றுக்கொள்ளப்போகிறீர்கள்:
1. **உரையாடல் வரலாற்றை குறைப்பது**: குறிப்புகளை பயன்படுத்துவதில் மேலாண்மை செய்ய உரையாடல் வரலாற்றை தானாக சுருக்குவது எப்படி
2. **ஏஜென்ட் ஸ்க்ராட்ச்பேட்**: பயனர் விருப்பங்கள் மற்றும் முடிக்கப்பட்ட பணிகளை கண்காணிக்க ஒரு நிலையான நினைவக அமைப்பு
3. **குறிப்புகள் பயன்பாட்டை கண்காணித்தல்**: வரலாற்றை குறைப்பதுடன் மற்றும் இல்லாமல் குறிப்புகள் பயன்பாட்டில் மாற்றங்களை கண்காணிக்க

## முன் தேவைகள்:
- சூழல் மாறிகள் அமைக்கப்பட்ட Azure OpenAI அமைப்பு
- முந்தைய பாடங்களில் இருந்து அடிப்படை ஏஜென்ட் கருத்துகளைப் புரிந்துகொள்வது


## தேவையான தொகுப்புகளை இறக்குமதி செய்யவும்


In [ ]:
# Import necessary packages
import json
import os
import asyncio
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv
from IPython.display import display, HTML, Markdown
from typing import Annotated, Optional

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.completion_usage import CompletionUsage
from semantic_kernel.contents import ChatHistorySummarizationReducer
from semantic_kernel.functions import kernel_function

## ஏஜென்ட் ஸ்க்ராட்ச்பேட் பற்றி புரிந்துகொள்வது

### ஏஜென்ட் ஸ்க்ராட்ச்பேட் என்றால் என்ன?

**ஏஜென்ட் ஸ்க்ராட்ச்பேட்** என்பது ஏஜென்ட்கள் பயன்படுத்தும் ஒரு நிலையான நினைவக அமைப்பு, இது:
- **முடிக்கப்பட்ட பணிகளை கண்காணிக்க**: பயனருக்காக என்ன செய்யப்பட்டது என்பதை பதிவு செய்ய
- **பயனர் விருப்பங்களை சேமிக்க**: விருப்பங்கள், விருப்பமற்றவை மற்றும் தேவைகளை நினைவில் வைத்திருக்க
- **சூழலமைப்பை பராமரிக்க**: முக்கியமான தகவல்களை உரையாடல்களில் எளிதாக அணுகக்கூடியதாக வைத்திருக்க
- **மீண்டும் மீண்டும் கேள்விகள் கேட்பதை தவிர்க்க**: ஒரே கேள்விகளை மீண்டும் கேட்காமல் இருக்க

### இது எப்படி செயல்படுகிறது:
1. **எழுதும் செயல்பாடுகள்**: ஏஜென்ட் புதிய தகவலைக் கற்றுக்கொண்ட பிறகு ஸ்க்ராட்ச்பேட்டில் புதுப்பிக்கிறது
2. **படிக்கும் செயல்பாடுகள்**: முடிவுகளை எடுக்கும் போது ஏஜென்ட் ஸ்க்ராட்ச்பேட்டை அணுகுகிறது
3. **நிலைத்தன்மை**: உரையாடல் வரலாறு குறைக்கப்பட்டாலும் தகவல் நீடிக்கிறது

இதை உரையாடல் வரலாற்றைத் துணை செய்யும் ஏஜென்டின் தனிப்பட்ட குறிப்பேடாகக் கருதலாம்.


## சூழல் கட்டமைப்பு


In [ ]:
# Load environment variables
load_dotenv()

# Create Azure OpenAI service
chat_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

print("✅ Azure OpenAI service configured")

## ஏஜென்ட் ஸ்கிராட்ச்பேட் பிளகின் உருவாக்கவும்

இந்த பிளகின் ஏஜென்டுக்கு நிலையான ஸ்கிராட்ச்பேட் கோப்பை படிக்கவும் எழுதவும் அனுமதிக்கிறது.


In [ ]:
class ScratchpadPlugin:
    """Plugin for managing agent scratchpad - a persistent memory for user preferences and completed tasks"""
    
    def __init__(self, filepath: str = "agent_scratchpad.md"):
        self.filepath = Path(filepath)
        # Initialize scratchpad if it doesn't exist
        if not self.filepath.exists():
            self.filepath.write_text("# Agent Scratchpad\n\n## User Preferences\n\n## Completed Tasks\n\n")
    
    @kernel_function(
        description="Read the current agent scratchpad to get user's travel preferences and completed tasks"
    )
    def read_scratchpad(self) -> Annotated[str, "The contents of the agent scratchpad"]:
        """Read the current scratchpad contents"""
        return self.filepath.read_text()
    
    @kernel_function(
        description="Update the agent scratchpad with new user's travel preference or completed tasks"
    )
    def update_scratchpad(
        self,
        category: Annotated[str, "Category to update: 'preferences' or 'tasks'"],
        content: Annotated[str, "The new content to add"]
    ) -> Annotated[str, "Confirmation of the update"]:
        """Update the scratchpad with new information"""
        current_content = self.filepath.read_text()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if category.lower() == "preferences":
            # Find the preferences section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## User Preferences" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        elif category.lower() == "tasks":
            # Find the tasks section and append
            lines = current_content.split("\n")
            for i, line in enumerate(lines):
                if "## Completed Tasks" in line:
                    lines.insert(i + 1, f"\n- [{timestamp}] {content}")
                    break
            current_content = "\n".join(lines)
        
        self.filepath.write_text(current_content)
        return f"✅ Scratchpad updated with {category}: {content}"

# Create the scratchpad plugin
scratchpad_plugin = ScratchpadPlugin("vacation_agent_scratchpad.md")
print("📝 Scratchpad plugin created")

## சந்தை வரலாற்றை குறைப்பதற்கான தொடக்கம்

ChatHistorySummarizationReducer, உரையாடல் வரலாறு ஒரு வரம்பை மீறும்போது தானாகவே சுருக்கமாக மாற்றுகிறது.


In [ ]:
# Configure reduction parameters
REDUCER_TARGET_COUNT = 5  # Target number of messages to keep after reduction
REDUCER_THRESHOLD = 15    # Trigger reduction when message count exceeds this

# Create the history summarization reducer
history_reducer = ChatHistorySummarizationReducer(
    service=chat_service,
    target_count=REDUCER_TARGET_COUNT,
    threshold_count=REDUCER_THRESHOLD,
)

print(f"🔄 Chat History Reducer configured:")
print(f"   - Reduction triggered at: {REDUCER_THRESHOLD} messages")
print(f"   - Reduces history to: {REDUCER_TARGET_COUNT} messages")

## விடுமுறை திட்டமிடும் முகவரியை உருவாக்கவும்

இந்த முகவர் பயனர்களுக்கு விடுமுறை திட்டமிட உதவுவதுடன், ஸ்கிராட்ச்பேடின் மூலம் சூழலை பராமரிக்கவும் செய்யும்.


In [ ]:
# Create the vacation planning agent with detailed instructions
agent = ChatCompletionAgent(
    service=chat_service,
    name="VacationPlannerAgent",
    instructions="""
    You are a helpful vacation planning assistant. Your job is to help users plan their perfect vacation.
    
    CRITICAL SCRATCHPAD RULES - YOU MUST FOLLOW THESE:
    1. FIRST ACTION: When starting ANY conversation, immedi ately call read_scratchpad() to check existing preferences
    2. AFTER LEARNING PREFERENCES: When user mentions ANY preference (destinations, activities, budget, dates), 
       immediately call update_scratchpad() with category 'preferences'
    3. AFTER COMPLETING TASKS: When you finish creating an itinerary or completing any task,
       immediately call update_scratchpad() with category 'tasks'
    4. BEFORE NEW ITINERARY: Always call read_scratchpad() before creating any itinerary
    
    EXAMPLES OF WHEN TO UPDATE SCRATCHPAD:
    - User says "I love beaches" → update_scratchpad('preferences', 'Loves beach destinations')
    - User says "budget is $3000" → update_scratchpad('preferences', 'Budget: $3000 per person for a week')
    - You create an itinerary → update_scratchpad('tasks', 'Created Bali itinerary for beach vacation')
    
    PLANNING PROCESS:
    1. Read scratchpad first
    2. Ask about preferences if not found
    3. Update scratchpad with new information
    4. Create detailed itineraries
    5. Update scratchpad with completed tasks
    
    BE EXPLICIT: Always announce when you're checking or updating the scratchpad.
    """,
    plugins=[scratchpad_plugin],
)

print("🤖 Vacation Planning Agent created with enhanced scratchpad instructions")

## உதவி செயல்பாடுகள் காட்சிக்கும் டோக்கன் கண்காணிப்பிற்கும்


In [ ]:
# Token tracking class
class TokenTracker:
    def __init__(self):
        self.history = []
        self.total_usage = CompletionUsage()
        self.reduction_events = []  # Track when reductions occur

    def add_usage(self, usage: CompletionUsage, message_num: int, thread_length: int = None):
        if usage:
            self.total_usage += usage
            entry = {
                "message_num": message_num,
                "prompt_tokens": usage.prompt_tokens,
                "completion_tokens": usage.completion_tokens,
                "total_tokens": usage.prompt_tokens + usage.completion_tokens,
                "cumulative_tokens": self.total_usage.prompt_tokens + self.total_usage.completion_tokens,
                "thread_length": thread_length
            }
            self.history.append(entry)

    def mark_reduction(self, message_num: int):
        self.reduction_events.append(message_num)

    def display_chart(self):
        """Display a chart showing token usage per message and the impact of reduction"""
        if not self.history:
            return

        html = "<div style='font-family: monospace; background: #2d2d2d; color: #f0f0f0; padding: 15px; border-radius: 8px; border: 1px solid #444;'>"
        html += "<h4 style='color: #4fc3f7; margin-top: 0;'>📊 Token Usage Analysis</h4>"
        html += "<pre style='color: #f0f0f0; margin: 0;'>"

        # Show prompt tokens per message to see reduction impact
        html += "<span style='color: #81c784;'>Prompt Tokens per Message (shows conversation context size):</span>\n"
        max_prompt = max(h["prompt_tokens"] for h in self.history)
        scale = 50 / max_prompt if max_prompt > 0 else 1

        for i, h in enumerate(self.history):
            bar_length = int(h["prompt_tokens"] * scale)
            bar = "█" * bar_length
            reduction_marker = " <span style='color: #ff6b6b;'>← REDUCTION!</span>" if h[
                "message_num"] in self.reduction_events else ""
            html += f"<span style='color: #aaa;'>Msg {h['message_num']:2d}:</span> <span style='color: #4fc3f7;'>{bar}</span> <span style='color: #ffd93d;'>{h['prompt_tokens']:,} tokens</span>{reduction_marker}\n"

        html += "\n</pre></div>"
        display(HTML(html))

        # Calculate reduction impact
        if self.reduction_events:
            # Find the message before and after first reduction
            first_reduction_msg = self.reduction_events[0]
            before_reduction = None
            after_reduction = None

            for h in self.history:
                if h["message_num"] == first_reduction_msg - 1:
                    before_reduction = h["prompt_tokens"]
                elif h["message_num"] == first_reduction_msg:
                    after_reduction = h["prompt_tokens"]

            if before_reduction and after_reduction:
                reduction_amount = before_reduction - after_reduction
                reduction_percent = (reduction_amount / before_reduction * 100)
                print(f"\n🔄 Actual Reduction Impact:")
                print(f"Prompt tokens before reduction: {before_reduction:,}")
                print(f"Prompt tokens after reduction: {after_reduction:,}")
                print(
                    f"Tokens saved: {reduction_amount:,} ({reduction_percent:.1f}%)")

# Display function for clean output


def display_message(role: str, content: str, color: str = "#2E8B57"):
    """Display a message with nice formatting that works in both light and dark themes"""
    # Use a semi-transparent background that adapts to the theme
    html = f"""
    <div style='
        margin: 10px 0; 
        padding: 12px 15px; 
        border-left: 4px solid {color}; 
        background: rgba(128, 128, 128, 0.1); 
        border-radius: 4px;
        color: inherit;
    '>
        <strong style='color: {color}; font-size: 14px;'>{role}:</strong><br>
        <div style='margin-top: 8px; white-space: pre-wrap; color: inherit; font-size: 14px;'>{content}</div>
    </div>
    """
    display(HTML(html))


# Initialize token tracker
token_tracker = TokenTracker()
print("📊 Token tracking initialized")

## விடுமுறை திட்டமிடல் உரையாடலை இயக்கவும்

இப்போது முழுமையான உரையாடலை இயக்கி பார்ப்போம், இது கீழ்கண்டவற்றை விளக்குகிறது:
1. ஆரம்ப திட்டமிடல் கோரிக்கை
2. விருப்பங்களை சேகரித்தல்
3. பயண திட்டம் உருவாக்குதல்
4. இடம் மாற்றம்
5. உரையாடல் வரலாற்றை குறைத்தல்
6. ஸ்கிராட்ச்பேட் பயன்பாடு


In [ ]:
# Define the conversation flow
user_inputs = [
    "I'm thinking about planning a vacation. Can you help me?",
    "I love beach destinations with great food and culture. I enjoy water sports, exploring local markets, and trying authentic cuisine. My budget is around $3000 per person for a week.",
    "That sounds perfect! Please create a detailed itinerary for Bali.",
    "Actually, I've changed my mind. I'd prefer to go to the Greek islands instead. Can you create a new itinerary?",
    "What's the weather like there?",
    "What should I pack?",
    "Are there any cultural customs I should know about?",
    "What's the best way to get around?"
]


async def run_vacation_planning():
    """Run the vacation planning conversation with token tracking and history reduction"""

    # Create thread with history reducer
    thread = ChatHistoryAgentThread(chat_history=history_reducer)
    message_count = 0
    scratchpad_operations = 0  # Track scratchpad usage

    print("🚀 Starting Vacation Planning Session\n")

    # Process conversation
    for i, user_input in enumerate(user_inputs):
        message_count += 1
        display_message("User", user_input, "#4fc3f7")  # Blue for user

        # Get agent response
        full_response = ""
        usage = None
        function_calls = []  # Track function calls

        async for response in agent.invoke(
            messages=user_input,
            thread=thread,
        ):
            if response.content:
                full_response += str(response.content)
            if response.metadata.get("usage"):
                usage = response.metadata["usage"]
            thread = response.thread

        display_message(f"{agent.name}", full_response,
                        "#81c784")  # Green for agent

        # Track tokens with thread length
        if usage:
            token_tracker.add_usage(usage, message_count, len(thread))

        # Check thread status and look for scratchpad operations
        print(f"📝 Thread has {len(thread)} messages")

        # Count scratchpad operations in this turn
        turn_scratchpad_ops = 0
        async for msg in thread.get_messages():
            if hasattr(msg, 'content') and msg.content:
                content_str = str(msg.content)
                if 'read_scratchpad' in content_str or 'update_scratchpad' in content_str:
                    turn_scratchpad_ops += 1

        if turn_scratchpad_ops > scratchpad_operations:
            print(
                f"   📝 Scratchpad operations detected: {turn_scratchpad_ops - scratchpad_operations} new operations")
            scratchpad_operations = turn_scratchpad_ops

        # Show message types for first message
        if i == 0:
            message_types = []
            async for msg in thread.get_messages():
                msg_type = msg.role.value if hasattr(
                    msg.role, 'value') else str(msg.role)
                message_types.append(msg_type)
            print(f"   Message types: {message_types[:10]}..." if len(
                message_types) > 10 else f"   Message types: {message_types}")

        # Check if reduction should happen
        if len(thread) > REDUCER_THRESHOLD:
            print(
                f"   ⚠️ Thread length ({len(thread)}) exceeds threshold ({REDUCER_THRESHOLD})")

            # Attempt reduction
            is_reduced = await thread.reduce()
            if is_reduced:
                print(
                    f"\n🔄 HISTORY REDUCED! Thread now has {len(thread)} messages\n")
                token_tracker.mark_reduction(message_count + 1)

                # Show summary if available
                async for msg in thread.get_messages():
                    if msg.metadata and msg.metadata.get("__summary__"):
                        display_message("System Summary", str(
                            msg.content), "#ff6b6b")
                        break

    # Display final token usage chart
    print("\n--- Token Usage Analysis ---")
    token_tracker.display_chart()

    # Show final scratchpad contents
    print("\n--- Final Scratchpad Contents ---")
    scratchpad_contents = scratchpad_plugin.read_scratchpad()
    display(Markdown(scratchpad_contents))

    print(f"\n📊 Total scratchpad operations: {scratchpad_operations}")

    return thread

# Run the conversation
thread = await run_vacation_planning()

## முடிவுகளை பகுப்பாய்வு செய்வது

நமது உரையாடலின் போது என்ன நடந்தது என்பதைப் பார்ப்போம்:


In [ ]:
# Analyze token usage
print("📊 Total Token Usage Summary\n")
print(f"Total Prompt Tokens: {token_tracker.total_usage.prompt_tokens:,}")
print(
    f"Total Completion Tokens: {token_tracker.total_usage.completion_tokens:,}")
print(
    f"Total Tokens Used: {token_tracker.total_usage.prompt_tokens + token_tracker.total_usage.completion_tokens:,}")

print("\n💡 Note: The reduction impact is shown in the chart above.")
print("Look for the dramatic drop in prompt tokens after the REDUCTION marker.")
print("This shows how chat history summarization reduces the context size for future messages.")

## முக்கிய குறிப்புகள்

### 1. உரையாடல் வரலாற்றை குறைத்தல்
- **தானியங்கியாக செயல்படுத்தல்**: செய்திகளின் எண்ணிக்கை வரம்பை மீறும்போது குறைப்பு நிகழ்கிறது
- **டோக்கன் சேமிப்பு**: சுருக்கத்திற்குப் பிறகு டோக்கன் பயன்பாட்டில் குறிப்பிடத்தக்க குறைப்பு
- **சூழல் பாதுகாப்பு**: முக்கியமான தகவல்கள் சுருக்கங்களில் பாதுகாக்கப்படுகின்றன

### 2. ஏஜென்ட் ஸ்கிராட்ச்பேட் நன்மைகள்
- **நிலையான நினைவகம்**: பயனர் விருப்பங்கள் வரலாற்று குறைப்பிற்குப் பிறகும் நிலைத்திருக்கின்றன
- **பணிகளை கண்காணித்தல்**: ஏஜென்ட் முடிக்கப்பட்ட பணிகளின் பதிவை பராமரிக்கிறது
- **மேம்பட்ட அனுபவம்**: விருப்பங்களை மீண்டும் கூற தேவையில்லை

### 3. டோக்கன் பயன்பாட்டு முறை
- **நேரியல் வளர்ச்சி**: ஒவ்வொரு செய்தியுடன் டோக்கன்கள் அதிகரிக்கின்றன
- **குறிப்பிடத்தக்க குறைவு**: குறைப்பு டோக்கன் எண்ணிக்கையை பெரிதும் குறைக்கிறது
- **நிலையான உரையாடல்கள்**: வரம்புகளுக்குள் நீண்ட உரையாடல்களை சாத்தியமாக்குகிறது


## சுத்தம் செய்யுதல்

இந்த டெமோவின் போது உருவாக்கப்பட்ட ஸ்கிராட்ச்பேட் கோப்பை சுத்தம் செய்யவும்:


In [ ]:
# Optional: Clean up the scratchpad file
# Uncomment the next line to delete the scratchpad
# Path("vacation_agent_scratchpad.md").unlink(missing_ok=True)

print("✅ Demo complete! The scratchpad file 'vacation_agent_scratchpad.md' has been preserved for your review.")

# சுருக்கம்

வாழ்த்துக்கள்! நீங்கள் மேம்பட்ட சூழல் மேலாண்மை திறன்களுடன் AI முகவரியை வெற்றிகரமாக செயல்படுத்தியுள்ளீர்கள்:

## நீங்கள் கற்றுக்கொண்டவை:
- **செய்தி வரலாற்றை சுருக்குதல்**: டோக்கன் வரம்புகளை நிர்வகிக்க உரையாடல்களை தானாக சுருக்குதல்
- **முகவர் ஸ்கிராட்ச்பேட்**: பயனர் விருப்பங்கள் மற்றும் முடிக்கப்பட்ட பணிகளுக்கான நிலையான நினைவகத்தை செயல்படுத்துதல்
- **டோக்கன் மேலாண்மை**: நீண்ட உரையாடல்களில் டோக்கன் பயன்பாட்டை கண்காணித்து மேம்படுத்துதல்
- **சூழல் பாதுகாப்பு**: உரையாடல் சுருக்கங்களில் முக்கிய தகவல்களை பராமரித்தல்

## நிஜ வாழ்க்கை பயன்பாடுகள்:
- **வாடிக்கையாளர் சேவை பாட்டுகள்**: அமர்வுகளுக்கு இடையில் வாடிக்கையாளர் விருப்பங்களை நினைவில் கொள்ளுதல்
- **தனிப்பட்ட உதவியாளர்கள்**: நடந்து கொண்டிருக்கும் திட்டங்கள் மற்றும் பயனர் பழக்கங்களை கண்காணித்தல்
- **கல்வி ஆசிரியர்கள்**: மாணவர் முன்னேற்றம் மற்றும் கற்றல் விருப்பங்களை பராமரித்தல்
- **சுகாதார உதவியாளர்கள்**: டோக்கன் வரம்புகளை மதிக்கும்போது நோயாளி வரலாற்றை வைத்திருத்தல்

## அடுத்த படிகள்:
- மேலும் மேம்பட்ட ஸ்கிராட்ச்பேட் ஸ்கீமாக்களை செயல்படுத்துதல்
- பல பயனர் சூழல்களுக்கு தரவுத்தொகுப்பு சேமிப்பை சேர்த்தல்
- துறைக்கு சிறப்பான தேவைகளுக்கான தனிப்பயன் சுருக்க உத்திகளை உருவாக்குதல்
- அர்த்தமான நினைவக தேடலுக்காக வெக்டர் தரவுத்தொகுப்புகளுடன் இணைத்தல்
- முழு சூழலுடன் நாட்களுக்கு பிறகு உரையாடல்களை மீண்டும் தொடங்கும் முகவர்களை உருவாக்குதல்



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியக்க மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
